# Preprocessing Only Workflow

In this workflow, CFRL takes in an offline trajectory and then preprocesses the offline trajectory 
using `SyntheticPreprocessor`. The final output of the workflow is the preprocessed (debiased) 
offline trajectory. This workflow is appropriate when the user does not want to train policies using 
CFRL. Instead, the user can take the preprocessed trajectory to train a counterfactually fair policy 
using another reinforcement learning library or application that better fits their needs.

We begin by importing the liberaries needed for this demonstration.

In [4]:
# Need this temporarily to import CFRL before it is officially published to PyPI
import sys
sys.path.append("E:/learning/university/MiSIL/CFRL Python Package/CFRL")

In [5]:
import pandas as pd
import numpy as np
#import pytorch as torch
from sklearn.model_selection import train_test_split
from CFRL.reader import read_trajectory_from_dataframe, convert_trajectory_to_dataframe
from CFRL.preprocessor import SequentialPreprocessor
np.random.seed(1) # ensure reproducibility
#torch.manual_seed(1)

## Data Loading

In this demonstration, we use an offline trajectory generated from a `SyntheticEnvironment` using some pre-specified transition rules. Although it is actually synthesized, we treat it as if it is from some unknown environment for pedagogical convenience in this demonstration.

The trajectory contains 500 individuals (i.e. $N=100$) and 10 transitions (i.e. $T=10$). The actions are binary (0 or 1) and were sampled using a random policy that selects 0 or 1 randomly with equal probability. It is stored in a tabular format in a `.csv` file. The sensitive attribute variable is bivariate, stored in columns `z1` and `z2`. The legit values of the sensitive attribute are $[0, 0]$, $[1, 0]$, $[0, 1]$, and $[1, 1]$. The state variable is tri-variate, stored in columns `state1`, `state2`, and `state3`. The actions are stored in the column `action` and rewards in the column `reward`. The tabular data also includes an extra inrrelevant column `timestamp`. 

We can load and view the tabular data.

In [6]:
trajectory = pd.read_csv('../data/sample_data_large_multi.csv')
trajectory

,Unnamed: 0,ID,timestamp,z1,z2,action,reward,state1,state2,state3
0,0,1.0,1.0,0.0,1.0,NaN,NaN,3.124345,0.888244,0.971828
1,1,1.0,2.0,0.0,1.0,1.0,7.380339,3.078124,3.695250,3.129721
2,2,1.0,3.0,0.0,1.0,0.0,12.299111,3.700923,3.088366,3.605864
3,3,1.0,4.0,0.0,1.0,0.0,10.933709,3.938980,4.468625,4.108137
4,4,1.0,5.0,0.0,1.0,1.0,14.626809,4.944344,4.277053,4.290724
...,...,...,...,...,...,...,...,...,...,...
5495,5495,500.0,7.0,1.0,0.0,0.0,16.140130,6.236726,3.673618,6.401583
5496,5496,500.0,8.0,1.0,0.0,0.0,17.709536,5.232273,5.330467,6.966696
5497,5497,500.0,9.0,1.0,0.0,0.0,17.863392,6.328855,6.008048,6.397695
5498,5498,500.0,10.0,1.0,0.0,0.0,18.654867,5.213347,5.948473,6.023897


We now read the trajectory from the tabular format into Trajectory Arrays.

In [7]:
zs, states, actions, rewards, ids = read_trajectory_from_dataframe(
                                                data=trajectory, 
                                                z_labels=['z1', 'z2'], 
                                                state_labels=['state1', 'state2', 'state3'], 
                                                action_label='action', 
                                                reward_label='reward', 
                                                id_label='ID', 
                                                T=10
                                                )

## Preprocessor Training

Before preprocessing the trajectory, we need to first train a preprocessor. To mitigate overfitting, we use a random subset of 250 individuals in the trajectory to train the preprocessor. The remaining 250 individuals will be actually preprocessed. We now form these two sets.

In [8]:
(
    zs_train, zs_prepro, 
    states_train, states_prepro, 
    actions_train, actions_prepro, 
    rewards_train, rewards_prepro, 
    ids_train, ids_prepro
) = train_test_split(zs, states, actions, rewards, ids, test_size=0.5)

We now use the training set to train a `SequentialPreprocessor`.

In [9]:
sp = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                            num_actions=2, 
                            cross_folds=1, 
                            mode='single', 
                            reg_model='nn')
sp.train_preprocessor(zs=zs_train, xs=states_train, actions=actions_train, rewards=rewards_train)

100%|██████████| 1000/1000 [00:26<00:00, 37.13it/s]


(array([[[ 0.29730609,  0.64565182, -1.68102797, ...,  2.32500351,
           2.80236241,  0.47934873],
         [-0.40034815, -1.30309663, -0.68382889, ...,  3.55263643,
           2.24261536,  2.89912063],
         [-1.62306811, -2.72240321, -2.00343044, ...,  3.39949302,
           2.34428819,  3.50777023],
         ...,
         [-1.22176325, -1.85526077, -1.32665902, ...,  9.88234269,
          10.51068981,  9.87003654],
         [-2.78983022, -1.20580641, -2.90492813, ...,  9.19271968,
          11.2210215 ,  9.56812938],
         [-2.41840585, -2.4534643 , -2.77835978, ..., 10.58553449,
          11.29762646, 10.01678216]],
 
        [[ 0.56773087,  0.6212389 ,  0.76884568, ...,  2.59542828,
           2.77794949,  2.92922237],
         [-0.15181686,  0.12701286, -1.36145366, ...,  3.52194039,
           3.51136637,  2.69899352],
         [-0.97777097, -2.61385588, -0.51860966, ...,  4.37716336,
           3.06812365,  5.2454351 ],
         ...,
         [-3.23892514, -1.9861518

## Data Preprocessing

We now preprocess the remaining data that are not in the training set.

In [10]:
states_tilde, rewards_tilde = sp.preprocess_multiple_steps(zs=zs_prepro, 
                                                           xs=states_prepro, 
                                                           actions=actions_prepro, 
                                                           rewards=rewards_prepro)

## Data Exporting

Finally, we convert the preprocessed trajectory back into the tabular format so that it is easier to store and manage. For simplicity, we call the new state as `state1`, ..., `state12`, which is the default option provided by `convert_trajectory_to_dataframe` (so we do not need to specify the `state_labels` argument).

In [11]:
preprocessed_trajectory = convert_trajectory_to_dataframe(
                                        zs=zs_prepro, 
                                        states=states_tilde, 
                                        actions=actions_prepro, 
                                        rewards=rewards_tilde, 
                                        ids=ids_prepro, 
                                        z_labels=['z1', 'z2'], 
                                        action_label='action', 
                                        reward_label='reward', 
                                        id_label='ID', 
                                        T_label='time_step'
                                        )
preprocessed_trajectory

,ID,time_step,z1,z2,action,reward,state1,state2,state3,state4,state5,state6,state7,state8,state9,state10,state11,state12
0,305.0,1.0,0.0,0.0,NaN,NaN,-0.545534,0.855285,1.255392,0.186795,1.741435,2.262210,0.501002,2.084589,2.099199,1.482163,3.011995,3.415769
1,305.0,2.0,0.0,0.0,0.0,5.904090,1.635313,1.973919,-1.833562,3.685160,4.198751,-0.452039,3.831087,4.448862,-0.265675,5.781165,5.682467,1.497074
2,305.0,3.0,0.0,0.0,1.0,8.461665,-0.449665,1.914116,-1.665698,2.106300,4.741234,1.140966,2.201151,4.952910,0.878392,4.419536,8.034381,3.297043
3,305.0,4.0,0.0,0.0,0.0,7.468761,-0.234639,-0.751243,-0.369390,3.048205,2.487832,3.436247,3.187059,2.734981,2.805609,6.122002,5.626643,6.166460
4,305.0,5.0,0.0,0.0,1.0,9.454961,0.927127,0.216693,-0.135216,5.199149,4.835119,3.107455,5.041436,4.715718,3.170032,8.701095,8.460822,7.010708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,473.0,7.0,1.0,1.0,1.0,17.367737,2.376917,0.853423,0.189816,7.375944,6.309883,5.551572,7.282390,6.090883,5.782331,12.283425,11.141589,10.667867
2746,473.0,8.0,1.0,1.0,0.0,17.304874,-0.906432,0.406786,0.035251,4.764992,6.226279,4.758157,5.057752,6.082017,5.239095,10.397726,11.400146,10.394255
2747,473.0,9.0,1.0,1.0,1.0,18.017587,-1.442580,-0.537270,0.009083,4.626450,5.242629,5.691870,4.755126,5.424193,6.025446,10.372732,11.143486,11.733089
2748,473.0,10.0,1.0,1.0,1.0,16.797489,-0.559398,-0.753199,-3.078257,5.131776,5.332984,3.206726,5.336041,5.533215,3.575978,11.350771,11.513769,9.569187


## Alternative: Preprocessing All Individuals

Sometimes, the number of individuals in the trajectory is small. In this case, if we only preprocess a subset of individuals, the resulting preprocessed trajectory might be too small to be useful. In this case, we can directly preprocess all individuals using the `train_preprocessor()` function when we have a relatively large number of `cross_folds`.

When `cross_folds=K` where `K` is greater than 1, `train_preprocessor()` will internally divide the training data into `K` folds. For each $i=1,\dots,k$, it trains a model based on all the folds other than the $i$-th one, which is then used to preprocess data in the $i$-th fold. This results in `K` folds of preprocessed data, each of which is processed using a model that is trained on the other folds. These `K` folds of data are then combined and returned by `train_preprocessor()`. This method allows us to preprocess all individuals in the trajectory while reducing overfitting.

To use this functionality, we first initialize a `SequentialPreprocessor` with `cross_folds` greater than 1. We use `cross_folds=5` here.

In [12]:
sp_cf5 = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                                num_actions=2, 
                                cross_folds=5, 
                                mode='single', 
                                reg_model='nn')

We now simultaneously train the preprocessor and preprocess all individuals in the trajectory using the precedure described above.

In [13]:
states_tilde_cf5, rewards_tilde_cf5 = sp_cf5.train_preprocessor(zs=zs, 
                                                                xs=states, 
                                                                actions=actions, 
                                                                rewards=rewards)

100%|██████████| 1000/1000 [00:36<00:00, 27.74it/s]


We can now convert the preprocessed trajectory into the tabular format.

In [14]:
preprocessed_trajectory_cf5 = convert_trajectory_to_dataframe(
                                            zs=zs_prepro, 
                                            states=states_tilde, 
                                            actions=actions_prepro, 
                                            rewards=rewards_tilde, 
                                            ids=ids_prepro, 
                                            z_labels=['z1', 'z2'], 
                                            action_label='action', 
                                            reward_label='reward', 
                                            id_label='ID', 
                                            T_label='time_step'
                                            )
preprocessed_trajectory_cf5

,ID,time_step,z1,z2,action,reward,state1,state2,state3,state4,state5,state6,state7,state8,state9,state10,state11,state12
0,305.0,1.0,0.0,0.0,NaN,NaN,-0.545534,0.855285,1.255392,0.186795,1.741435,2.262210,0.501002,2.084589,2.099199,1.482163,3.011995,3.415769
1,305.0,2.0,0.0,0.0,0.0,5.904090,1.635313,1.973919,-1.833562,3.685160,4.198751,-0.452039,3.831087,4.448862,-0.265675,5.781165,5.682467,1.497074
2,305.0,3.0,0.0,0.0,1.0,8.461665,-0.449665,1.914116,-1.665698,2.106300,4.741234,1.140966,2.201151,4.952910,0.878392,4.419536,8.034381,3.297043
3,305.0,4.0,0.0,0.0,0.0,7.468761,-0.234639,-0.751243,-0.369390,3.048205,2.487832,3.436247,3.187059,2.734981,2.805609,6.122002,5.626643,6.166460
4,305.0,5.0,0.0,0.0,1.0,9.454961,0.927127,0.216693,-0.135216,5.199149,4.835119,3.107455,5.041436,4.715718,3.170032,8.701095,8.460822,7.010708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,473.0,7.0,1.0,1.0,1.0,17.367737,2.376917,0.853423,0.189816,7.375944,6.309883,5.551572,7.282390,6.090883,5.782331,12.283425,11.141589,10.667867
2746,473.0,8.0,1.0,1.0,0.0,17.304874,-0.906432,0.406786,0.035251,4.764992,6.226279,4.758157,5.057752,6.082017,5.239095,10.397726,11.400146,10.394255
2747,473.0,9.0,1.0,1.0,1.0,18.017587,-1.442580,-0.537270,0.009083,4.626450,5.242629,5.691870,4.755126,5.424193,6.025446,10.372732,11.143486,11.733089
2748,473.0,10.0,1.0,1.0,1.0,16.797489,-0.559398,-0.753199,-3.078257,5.131776,5.332984,3.206726,5.336041,5.533215,3.575978,11.350771,11.513769,9.569187
